In [ ]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import re
import seaborn as sns
import QuantLib as ql

from bgs.load_bgs_amounts import load_bgs_amounts
from bgs.load_gilt_details import load_csv_blocks
from bgs.load_bgs_prices import load_prices
from bgs.gilt_analytics import gilt_yield, yield_series
from bgs.linker_analytics import linker_real_yield

In [ ]:
def clean_date(bgs_index):
    if re.match(r'^\d{2}\s\w{3}\s+\d{4}$', bgs_index):
        return pd.to_datetime(bgs_index, format="%d %b %Y", errors='coerce')
    else:
        return pd.to_datetime(bgs_index)

def clean_percentage(x):
    if x.strip() in ['Variable','Floating']:
        return x
    try:
        x = float(x)
    except (ValueError, TypeError):
        units, fractions = x.split(' ')
        num, denom = map(float, fractions.split('/'))
        x = float(units) + num / denom
        print(x)
    return x

### British Gilt Survey Amounts Table

In [ ]:
tables = load_bgs_amounts("downloads/BGSAmounts.csv")

conv = tables['Conventionals']
conv.replace("Redeemed", "", inplace=True)
conv = conv.apply(pd.to_numeric, errors='coerce')
conv.fillna(0, inplace=True)
conv.index = pd.Index([clean_date(x) for x in list(conv.index)]).to_period('M').to_timestamp('M')
conv.columns = [np.int64(x.strip()) for x in conv.columns]

old = tables['Calculated indexed nominal Old-style']
old.replace("Redeemed", "", inplace=True)
old = old.apply(pd.to_numeric, errors='coerce')
old.fillna(0, inplace=True)
old.index = pd.Index([clean_date(x) for x in list(old.index)]).to_period('M').to_timestamp('M')
old.columns = [np.int64(x.strip()) for x in old.columns]

new = tables['Calculated indexed nominal New-style']
new.replace("Redeemed", "", inplace=True)
new = new.apply(pd.to_numeric, errors='coerce')
new.fillna(0, inplace=True)
new.index = pd.Index([clean_date(x) for x in list(new.index)]).to_period('M').to_timestamp('M')
new.columns = [np.int64(x.strip()) for x in new.columns]

new_no_idx = tables['Index-linked New-style']
new_no_idx.replace("Redeemed", "", inplace=True)
new_no_idx = new_no_idx.apply(pd.to_numeric, errors='coerce')
new_no_idx.fillna(0, inplace=True)
new_no_idx.index = pd.Index([clean_date(x) for x in list(new_no_idx.index)]).to_period('M').to_timestamp('M')
new_no_idx.columns = [np.int64(x.strip()) for x in new_no_idx.columns]

### British Gilts Survey Month End Prices

In [ ]:
price_df=load_prices("downloads/BGSPrices.csv")
price_df.index = pd.to_datetime(price_df.index, format="%d %b %Y").to_period('M').to_timestamp('M')
for text in ['Amalgamated', 'Redeemed', 'redeemed']:
    price_df = price_df.replace(text, 0)
price_df = price_df.replace('missing', None)
price_df = price_df.ffill(axis=0)
price_df = price_df.fillna(0)
price_df.columns = [np.int64(x) for x in price_df.columns]

### British Gilts Survey Details Table (Bond Static Data, Coupon Maturity etc.) 

In [ ]:
details = load_csv_blocks("downloads/BGSDetails.csv")

conv_details = details['Conventionals']
conv_details['%'] = conv_details['%'].apply(clean_percentage)
conv_details['Sequence'] = conv_details['Sequence'].apply(lambda x: np.int64(x))

new_details = details['Index-Linked New-style']
new_details['%'] = new_details['%'].apply(clean_percentage)
new_details['Sequence'] = new_details['Sequence'].apply(lambda x: np.int64(x))

old_details = details['Index-Linked Old-style']
old_details['%'] = old_details['%'].apply(clean_percentage)
old_details['Sequence'] = old_details['Sequence'].apply(lambda x: np.int64(x))

date_variables = ["Latest redemption date","Issue date","First coupon payable on date",]
for date_var in date_variables:
    conv_details[date_var] = conv_details[date_var].apply(clean_date)
    old_details[date_var] = old_details[date_var].apply(clean_date)
    new_details[date_var] = new_details[date_var].apply(clean_date)


# ONS Nominal GDP

In [ ]:
# https://www.ons.gov.uk/economy/grossdomesticproductgdp/timeseries/ybha/pn2
ons_download_ybha = "downloads/series-020925.csv"
df_ybha = pd.read_csv(ons_download_ybha, skiprows=8,names=["CDID","YBHA"])
q_start =  df_ybha.loc[df_ybha['CDID']=="1955 Q1"].index[0]
df_ybha_q = df_ybha.iloc[q_start:]
df_ybha_q['date']=df_ybha_q['CDID'].apply(lambda x: f"{x.split(' Q')[0]}-{int(x.split(' Q')[1])*3}-01")
df_ybha_q['date']=pd.to_datetime(df_ybha_q['date'], format='%Y-%m-%d')
df_ybha_q.set_index('date', inplace=True)
df_ybha_m = df_ybha_q.resample('M').ffill()
df_ybha_m.head(10)

In [ ]:
# def get_ons_data(code):
#     user_agent = "econ-project/Version1.0.0 (organisation-name contact:cormach@github)"
#     url = "https://api.beta.ons.gov.uk/v1"
#     headers = {
#         "User-Agent": user_agent
#     }
#     response = requests.get(f"https://api.beta.ons.gov.uk/v1/economy/grossdomesticproductgdp/timeseries/ybha", headers=headers)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         response.raise_for_status()

# r = get_ons_data("YBHA")

### ONS Net Debt (excluding public sector banks) as a % of GDP: NSA (PUSF)

In [ ]:
ons_download_pusf = "downloads/series-030925.csv"
df_pusf = pd.read_csv(ons_download_pusf, skiprows=8,names=["CDID","HF6X"])
q_start =  df_pusf.loc[df_pusf['CDID']=="1975 Q1"].index[0]
q_end =  df_pusf.loc[df_pusf['CDID']=="2025 Q2"].index[0]
df_pusf_q = df_pusf.iloc[q_start:q_end].ffill()
df_pusf_q['date']=df_pusf_q['CDID'].apply(lambda x: f"{x.split(' Q')[0]}-{int(x.split(' Q')[1])*3}-01")
df_pusf_q['date']=pd.to_datetime(df_pusf_q['date'], format='%Y-%m-%d')
df_pusf_q.set_index('date', inplace=True)
df_pusf_m = df_pusf_q.resample('ME').ffill()
df_pusf_m.head(10)
df_pusf_m['HF6X'] = df_pusf_m['HF6X'].apply(lambda x: x/100)

In [ ]:
df_pusf_q

In [ ]:
# price_df.columns = [str(x) for x in price_df.columns]

In [ ]:
mtm_list = []

for bgs_gilt_id in conv.columns:
    # print(bgs_gilt_id)
    calc_df = pd.DataFrame(index=price_df.index, columns=['amounts','prices'])
    calc_df = pd.concat([conv[bgs_gilt_id],price_df[bgs_gilt_id]], join='inner', axis=1, keys=['amounts','prices'])
    calc_df['mtm'] = calc_df['amounts']*calc_df['prices']/100
    mtm_list.append(calc_df['mtm'].rename(bgs_gilt_id).copy())
#mtm_df[bgs_gilt_id] = calc_df['amounts']*calc_df['prices']
#mtm_list.append(mtm_df)
conv_mtm_df = pd.concat(mtm_list, axis=1)

conventionals = pd.DataFrame()
conventionals['Nominal'] = conv.sum(axis=1)
conventionals['MTM'] = conv_mtm_df.sum(axis=1)
conventionals.index = pd.to_datetime(conventionals.index, format="%b %Y").to_period('M').to_timestamp('M')

conventionals.plot()

In [ ]:
(conventionals['Nominal'] - conventionals['MTM']).plot()

In [ ]:
conventionals['2021-09-01':].plot()

In [ ]:
for bgs_gilt_id in old.columns:
    # print(bgs_gilt_id)
    calc_df = pd.DataFrame(index=price_df.index, columns=['amounts','prices'])
    calc_df = pd.concat([old[bgs_gilt_id],price_df[bgs_gilt_id]], join='inner', axis=1, keys=['amounts','prices'])
    calc_df['mtm'] = calc_df['amounts']*calc_df['prices']/100
    mtm_list.append(calc_df['mtm'].rename(bgs_gilt_id).copy())

In [ ]:
for bgs_gilt_id in new.columns:
    # print(bgs_gilt_id)
    calc_df = pd.DataFrame(index=price_df.index, columns=['amounts','prices'])
    calc_df = pd.concat([new[bgs_gilt_id],price_df[bgs_gilt_id]], join='inner', axis=1, keys=['amounts','prices'])
    calc_df['mtm'] = calc_df['amounts']*calc_df['prices']/100
    mtm_list.append(calc_df['mtm'].rename(bgs_gilt_id).copy())

In [ ]:
total_gilts_mtm = pd.concat(mtm_list, axis=1)

total_gilts = pd.DataFrame()
total_gilts['MTM'] = total_gilts_mtm.sum(axis=1)
total_gilts.index = pd.to_datetime(total_gilts.index, format="%b %Y").to_period('M').to_timestamp('M')


In [ ]:
total_nominal = tables['Sum of total conventional and indexed-linked'].reset_index().set_index('index')
total_nominal = total_nominal.drop_duplicates()
total_nominal.index = pd.Index([clean_date(x) for x in list(total_nominal.index)]).to_period('M').to_timestamp('M')

total_nominal['Sum of total conventional and indexed-linked'] = pd.to_numeric(total_nominal['Sum of total conventional and indexed-linked'])

In [ ]:
total_nominal['conv']= conventionals['Nominal']

In [ ]:
def before_linkers(row):
    if row['Sum of total conventional and indexed-linked'] == "":
        total_nominal =  row['conv']
    else:
        total_nominal =  row['Sum of total conventional and indexed-linked']
    return total_nominal

total_nominal['Total Nominal'] = total_nominal.apply(before_linkers, axis=1)

In [ ]:
# ax = total_nominal['Sum of total conventional and indexed-linked'].plot()
# total_gilts.plot(ax = ax)
total_gilts['Nominal']=total_nominal['Total Nominal']

In [ ]:
total_gilts.head()

In [ ]:
total_gilts = total_gilts.merge(df_ybha_m, left_index=True, right_index=True, how='inner')

In [ ]:
total_gilts.tail()

In [ ]:
total_gilts['Debt to GDP'] = total_gilts['Nominal'] / (total_gilts['YBHA'] * 4)
total_gilts['MTM to GDP'] = total_gilts['MTM'] / (total_gilts['YBHA'] * 4)

In [ ]:
ax = total_gilts[['Debt to GDP','MTM to GDP']].plot()
df_pusf_m.plot(ax=ax)

In [ ]:
ax = total_gilts['2021-09-01':][['Debt to GDP','MTM to GDP']].plot()
df_pusf_m['2021-09-01':].plot(ax=ax)

### Illustrating Bond Price Moves

In [ ]:
id = 32280
bond = conv_details[conv_details['Sequence']==id]
issue_date=bond['Issue date'].dt.strftime("%Y-%m-%d").values[0]
maturity_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
first_cpn_date=bond['First coupon payable on date'].dt.strftime("%Y-%m-%d").values[0]
last_cpn_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
coupon=bond['%'].values[0]
bond_price_series = price_df[id].replace(0, np.nan).copy().dropna()

ukt_4_2060 = yield_series(bond_price_series, issue_date, maturity_date, first_cpn_date, last_cpn_date, coupon)


In [ ]:
ukt_4_2060['yield'].plot()

### The state of play at OBR March 2025

In [ ]:
last = '2025-03-31'

In [ ]:
outstanding = conv.loc[last, conv.loc[last] !=0].reset_index()['index'].to_list()
outstanding[:10]

In [ ]:
last_price = price_df.loc[last][outstanding]
principal = conv.loc[last][outstanding]

In [ ]:
principal.name = 'balance_outstanding'
last_price.name = 'price'

In [ ]:
data_set_0325 = last_price.reset_index().merge(principal.reset_index(), on='index')
data_set_0325.rename(columns={'index': 'Sequence'}, inplace=True)

In [ ]:
(data_set_0325['balance_outstanding']*data_set_0325['price']).sum()/data_set_0325['balance_outstanding'].sum()

In [ ]:
(data_set_0325['balance_outstanding']*0.17).sum()

In [ ]:
data_set_0325.head()

In [ ]:
conv_details['Sequence'] = conv_details['Sequence'].astype(int)

In [ ]:
len([x for x in conv_details['Sequence'] if x in [y for y in outstanding]])

In [ ]:
conv_details.columns

In [ ]:
conv_details.loc[0, 'Sequence']

In [ ]:
data_set_0325.loc[0, 'Sequence']

In [ ]:
static_variables = ['%', "Frequency","Sequence",'ISIN Code']+date_variables
data_set_0325 = data_set_0325.merge(conv_details[static_variables], on='Sequence', how='inner')
data_set_0325.head()

In [ ]:
(data_set_0325['balance_outstanding']*data_set_0325['%']).sum()/data_set_0325['balance_outstanding'].sum()

In [ ]:
data_set_0325['balance_outstanding'].plot(kind='hist')

In [ ]:
sns.set_theme(style="darkgrid")

# use the scatterplot function to build the bubble map
ax = sns.scatterplot(
    data=data_set_0325,
    x="Latest redemption date",
    y="price",
    size="balance_outstanding",
    legend=False,
    sizes=(10, 500)
)

# show the graph
plt.show()

In [ ]:
range = ax.get_xaxis().get_data_interval()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.svm import SVR

pipeline = make_pipeline(
    preprocessing.StandardScaler(),
    SVR(kernel='linear', epsilon=0.01, C=100, gamma = 0.01),
)

X = ((data_set_0325['Latest redemption date'] - pd.Timestamp("1970-01-01")).dt.days).to_list()
# X = [np.array(X)[:, np.newaxis]]
X = np.array(X).reshape(-1, 1)
y = data_set_0325['price']

pipeline.fit(X, y)

In [ ]:
y_hat = pipeline.predict(X)

In [ ]:
sns.set_theme(style="darkgrid")

# use the scatterplot function to build the bubble map
ax = sns.scatterplot(
    data=data_set_0325,
    x="Latest redemption date",
    y="price",
    size="balance_outstanding",
    legend=False,
    sizes=(10, 500)
)

# show the graph
ax.plot(data_set_0325['Latest redemption date'], y_hat)
plt.show()

In [ ]:
decision_line = pd.DataFrame(
    {
        "Latest redemption date": data_set_0325['Latest redemption date'],
        "decision_line_price": y_hat
    })

In [ ]:
data_set_0325 = data_set_0325.merge(decision_line, on='Latest redemption date', how='inner')

In [ ]:
(
    (data_set_0325['decision_line_price'] -
     data_set_0325['price'])/data_set_0325['price']
     ).plot(kind='hist', bins=50, title='Difference between decision line and actual price')

In [ ]:
sns.set_theme(style="darkgrid")

f, axs = plt.subplots(1, 1, figsize=(8, 4))

condition = (data_set_0325['decision_line_price'] - data_set_0325['price']) > 2
data_set_0325['segmentation'] = condition.apply(lambda x: 'low coupon' if x else 'high coupon')

# use the scatterplot function to build the bubble map
sns.scatterplot(
    data=data_set_0325,
    x="Latest redemption date",
    y="price",
    size="balance_outstanding",
    sizes=(10, 500),
    hue='segmentation',
    legend='brief',
    ax=axs
)

# show the graph
axs.plot(data_set_0325['Latest redemption date'], y_hat, )
sns.move_legend(axs, "upper left", bbox_to_anchor=(1, 1))
f.show()